In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
import pickle as pkl

In [4]:
%run utils.ipynb
%run targets.ipynb

In [5]:
# Get Data
df = pd.read_csv('data/ETHBTC_1m_1519496760000_1549568220000.csv')
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [6]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE
0  1519496760000  0.085541  0.085570  0.085407  0.085473
1  1519496820000  0.085473  0.085570  0.085448  0.085449
2  1519496880000  0.085448  0.085569  0.085400  0.085538
3  1519496940000  0.085538  0.085546  0.085402  0.085540
4  1519497000000  0.085539  0.085552  0.085464  0.085464

In [7]:
data_validation(data)

In [8]:
# Get Target
%run targets.ipynb
target = get_target_1()
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [9]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape

(499999, 6)

In [10]:
data_validation(data)

In [19]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299999, 6)

(100000, 6)

(100000, 6)

In [25]:
# Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = scaler.fit(TR_data.iloc[:, 1:-1].values)

TR_data.iloc[:, 1:-1] = scaler.transform(TR_data.iloc[:, 1:-1])
CV_data.iloc[:, 1:-1] = scaler.transform(CV_data.iloc[:, 1:-1])
TT_data.iloc[:, 1:-1] = scaler.transform(TT_data.iloc[:, 1:-1])

In [35]:
win=120
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=win, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=win, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=win, tt_win=0, point_target=True)

In [36]:
TR_X.shape
TR_y.shape

(299880, 481)

(299880,)

In [37]:
%run hyperopt.ipynb
%run models.ipynb

hyperopt = Hyperopt(BayesR())
hyperopt.run(TR_X[:, 1:], TR_y, eval_set={
    'CV': (CV_X[:, 1:], CV_y),
    'TT': (TT_X[:, 1:], TT_y),
    'TR': (TR_X[:, 1:], TR_y)
})

{'lambda_2': 1e-07, 'alpha_2': 1e-05, 'lambda_1': 1e-05, 'tol': 0.001, 'alpha_1': 1e-07}
Convergence after  44  iterations
         acc       auc        f1   log_loss
TT  0.515744  0.501127  0.085464  16.725637
TR  0.520248  0.500000  0.000000  16.570044
CV  0.523913  0.501824  0.071685  16.443461
{'lambda_2': 1e-05, 'alpha_2': 1e-05, 'lambda_1': 1e-05, 'tol': 0.0001, 'alpha_1': 1e-06}
Convergence after  48  iterations
         acc       auc        f1   log_loss
TT  0.501166  0.503839  0.532432  17.229342
TR  0.525233  0.508986  0.178843  16.397894
CV  0.495740  0.497669  0.504037  17.416753
{'lambda_2': 1e-07, 'alpha_2': 1e-05, 'lambda_1': 1e-06, 'tol': 0.0001, 'alpha_1': 1e-07}
Convergence after  48  iterations
         acc       auc        f1   log_loss
TT  0.501166  0.503839  0.532432  17.229342
TR  0.525233  0.508986  0.178843  16.397894
CV  0.495740  0.497669  0.504037  17.416753
{'lambda_2': 1e-06, 'alpha_2': 1e-05, 'lambda_1': 1e-05, 'tol': 0.001, 'alpha_1': 1e-05}
Convergence 

KeyboardInterrupt: 

In [ ]:
best_itr = np.argmax(hyperopt.hp_trials.losses())
best_score = hyperopt.hp_trials.results[best_itr]['loss']
best_params = hyperopt._model.resolve_params(hyperopt.hp_trials.results[best_itr]['params'])

best_itr
best_score
best_params

In [29]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge(**best_params)
model = model.fit(TR_X[:, 1:], TR_y)

Convergence after  30  iterations


In [30]:
TR_y_pred = model.predict(TR_X[:, 1:])
TR_y_pred[:5]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TR_metrics

array([0.48279648, 0.4868741 , 0.48149322, 0.46780731, 0.49075502])

{'acc': 0.5251950390078015,
 'auc': 0.5090000659415749,
 'f1': 0.18104970183499428,
 'log_loss': 16.39922040203561}

In [31]:
(TR_y_pred < 0.5).sum()

269935

In [32]:
CV_y_pred = model.predict(CV_X[:, 1:])
CV_y_pred[:5]

CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics

array([11.85229699,  1.08994633, -1.09752468,  8.77987702,  8.10054757])

{'acc': 0.49402147266887464,
 'auc': 0.4968271345302028,
 'f1': 0.511108532977551,
 'log_loss': 17.476114588553667}

In [33]:
TT_y_pred = model.predict(TT_X[:, 1:])
TT_y_pred[:5]

TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics

array([ 1.29661498,  1.9337201 ,  2.69981702,  1.78254761, -0.6549673 ])

{'acc': 0.5023163666563273,
 'auc': 0.5053044203430431,
 'f1': 0.5375654292062961,
 'log_loss': 17.189626084508035}

In [34]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL', 
    'win':win
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': 'BayesR',
    'n_iter':30, 
    'accuracy':'{:.4f}'.format((CV_metrics['acc']+TT_metrics['acc'])/2)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('models/'+time_tag+'.pkl', 'wb')
pkl.dump(model, f, -1)
f.close()

%notebook -e models/{big_tag}.ipynb